In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from sqlalchemy import create_engine
import re

In [2]:
path = r'T:\MPO\TIP\TIP FY24-27\Maps\Data'

In [3]:
file = path + '\\MTIP24_27_List.xlsx'

In [4]:
data = pd.read_excel(file)

In [5]:
data.columns

Index(['Geo', 'Project Name', 'MTIP ID #', 'Project Description', 'Work Type',
       'Perf. Meas.', 'RTP Project Number / Ref.', 'In AQ CATS?', 'In\nAQMA?',
       'Air Quality Status', 'STIP Key', 'FFY', 'Phase', 'Phase Status',
       'Federal Funding ', 'Federal Funding Source', 'Federal Req. Match',
       'Federal Req. Match Source', 'Total Fed+ Req Match', 'Other Funding',
       'Other Funding Source', 'Total All Sources'],
      dtype='object')

In [6]:
data['Air Quality Status'].unique()

array(['EXEMPT / Other-Planning and Technical Studies (IAC conf 4/26/20)',
       nan, 'EXEMPT / Other-Planning and Technical Studies',
       'N/A (IAC conf 4/26/20)', 'N/A (IAC conf 11/3/21)',
       'EXEMPT / Air Quality - Bicycle and Pedestrian facilities',
       'Outside PM10 air quality maintenance area',
       'Outside PM10 air quality maintenance area (IAC conf 11/3/21)',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation',
       'EXEMPT / Other - Specific activities which do not involve or lead directly to construction',
       'EXEMPT / Other - Planning and Technical Studies',
       'EXEMPT / Safety - Projects that correct, improve, or eliminate a hazardous location or feature',
       'Part of K16223, project level conformity was approved in interagency meeting 12/23/2020',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation; Air Quality - Bicycle and Pedestrian facilities (IAC conf 4/26/20)',
       'EXEMPT / Safety - Pro

In [7]:
def getAQvars(x, var=['AQ Exempt?', 'AQ Status', 'IAC']):
    if str(x) == 'nan':
        res = None
    elif var == 'AQ Exempt?':
        if 'EXEMPT' in x:
            res = 'Yes'
        else:
            res = 'No'
    
    elif var == 'IAC':
        if 'IAC' in x:
            res = 'IAC ' + x.split(' (IAC ')[1].split(')')[0]
        else:
            res = 'N/A'
    
    elif var == 'AQ Status':
        if 'EXEMPT' in x:
            if 'IAC' in x:
                res = x.split('EXEMPT / ')[1].split(' (IAC')[0]
            else:
                res = x.split('EXEMPT / ')[1]
        elif 'IAC' in x:
            res = x.split(' (IAC')[0]
        else:
            res = x
    
    return res

In [8]:
data['AQ Exempt?'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Exempt?')) 

In [9]:
data['AQ Status'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Status')) 

In [10]:
data['IAC'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='IAC')) 

In [11]:
# this data has been modified in ArcGIS Pro after mapping
points = gpd.read_file(os.path.join(path, 'FY24_27_points.shp'))

In [12]:
# this data has been modified in ArcGIS Pro after mapping
lines = gpd.read_file(os.path.join(path, 'FY24_27_lines.shp'))

In [13]:
# read data from RLIDgeo
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [14]:
rep_sql = '''
SELECT 
repdist AS id,
repname AS name,
Shape.STAsBinary() AS geometry
FROM dbo.StateRepDist;
'''

In [15]:
sen_sql = '''
SELECT 
sendist AS id,
senname AS name,
Shape.STAsBinary() AS geometry
FROM dbo.StateSenDist;
'''

In [16]:
StateRepDist = gpd.GeoDataFrame.from_postgis(rep_sql, engine, geom_col='geometry')

In [17]:
StateRepDist.crs = "EPSG:2914"

In [18]:
StateSenDist = gpd.GeoDataFrame.from_postgis(sen_sql, engine, geom_col='geometry') 

In [19]:
StateSenDist.crs = "EPSG:2914"

In [20]:
StateRepDist = StateRepDist.to_crs(epsg=2992)

In [21]:
StateSenDist = StateSenDist.to_crs(epsg=2992)

In [22]:
# points in polygons
def get_pip(points, polygon):
    id_list = list(polygon.id)
    df = pd.DataFrame().reindex_like(points).dropna()
    for ID in id_list:
        pol = (polygon.loc[polygon.id==ID])
        pol.reset_index(drop = True, inplace = True)
        pip_mask = points.within(pol.loc[0, 'geometry'])
        pip_data = points.loc[pip_mask].copy()
        pip_data['id']= ID
        df = df.append(pip_data)
    df.reset_index(inplace=True, drop=True)
    df = df.drop(columns='geometry')
    return df

In [23]:
def reorder(x):
    if len(x) >= 4:
        if ',' in re.search(r',', x).group():
            res = ', '.join(str(x) for x in sorted([eval(i) for i in x.split(', ')]))
        else:
            res = x
    else:
        res = x
    return res

In [24]:
def get_district_IDs(dat=points,
                     shptype='point',
                     ply1=StateSenDist, 
                     ply2=StateRepDist, 
                     keycol='STIP_Key',
                     colnm1='Senator District',
                     colnm2='Representative District'):
    if shptype=='point':
        joined1 = get_pip(dat, ply1)
        joined2 = get_pip(dat, ply2)
    
    else:
        joined1 = gpd.sjoin(dat, ply1)
        joined2 = gpd.sjoin(dat, ply2)    
    
    joined1.rename(columns={'id':colnm1}, inplace=True)
    joined2.rename(columns={'id':colnm2}, inplace=True)

    joined1.drop_duplicates(ignore_index=True, inplace=True)
    joined2.drop_duplicates(ignore_index=True, inplace=True)

    joined1m = joined1[[keycol, colnm1]].groupby(keycol)[colnm1].apply(', '.join).reset_index()
    joined2m = joined2[[keycol, colnm2]].groupby(keycol)[colnm2].apply(', '.join).reset_index()
    joined_c = joined1m.merge(joined2m, on=keycol)
    
    joined_c[colnm1] = joined_c[colnm1].apply(reorder)
    joined_c[colnm2] = joined_c[colnm2].apply(reorder)
    
    return joined_c

In [25]:
pointdata = get_district_IDs()

In [26]:
linedata = get_district_IDs(dat=lines, shptype='line')

In [27]:
df = pointdata.append(linedata , ignore_index=True)

In [28]:
df.rename(columns={'STIP_Key':'STIP Key'}, inplace=True)

In [29]:
df['STIP Key'] = df['STIP Key'].astype(np.int64)

In [30]:
data.shape

(180, 25)

In [31]:
ndata = data.merge(df, how='outer', on='STIP Key')

In [32]:
ndata.shape

(188, 27)

In [33]:
ndata.to_excel(os.path.join(path, 'MTIP24_27_List_Updated.xlsx'), 
               sheet_name='2024-2027 Project List', 
               index=False)  